In [1]:
import os
import numpy as np
import tensorflow as tf
from keras import models, layers

from loop import TrainingLoop
from batch_selection import windowed_batch_selector, sorting_batch_selector

In [2]:
# Loading the dataset from the files saved in the preprocessing notebook.
path = 'data/iris'
prefix = 'iris_'
X_train = np.load(os.path.join(path, prefix+'train_vectors.npy'))
y_train = np.load(os.path.join(path, prefix+'train_labels.npy'))
X_test  = np.load(os.path.join(path, prefix+'test_vectors.npy'))
y_test  = np.load(os.path.join(path, prefix+'test_labels.npy'))

In [3]:
def build_model():
    # Setting up the model.
    model = models.Sequential()
    model.add(layers.Dense(50, activation='relu', input_shape=(X_test.shape[1],)))
    model.add(layers.Dense(40, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))
    return model

In [4]:
def train(model, X_train, y_train, batch_selector, epochs):
    selector_str = {windowed_batch_selector: 'windowed', sorting_batch_selector: 'sorting', None: 'original'}
    print('\n\n'+selector_str[batch_selector]+'\n')
    # Put the model in our custom training loop.
    TrainingLoop(
        model, 
        X_train, 
        y_train, 
        validation_split = 0.1,
        batch_size = 4,
        optimizer = tf.keras.optimizers.Adam(),
        loss_function = tf.keras.losses.CategoricalCrossentropy(),
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        batch_selection = batch_selector,
        log_file = os.path.join('logs',selector_str[batch_selector],'iris.csv')
    ).train(epochs)  # Training the model.

In [5]:
def evaluate(model, X_test, y_test):
    # We still have to compile the model for the test evaluation.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # After compiling we can run the evaluation.
    model.evaluate(X_test, y_test)

In [6]:
# Train and evaluate models with different batch selection algorithms.
for batch_selector in [None, windowed_batch_selector, sorting_batch_selector]:
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model = build_model()
    train(model, X_train, y_train, batch_selector=batch_selector, epochs=20)
    evaluate(model, X_test, y_test)



original

Epoch 1/20	Loss: 0.7039	Metrics: 0.4351: 	Validation metrics: 0.4166: 	100% | 27/27 [00:03<00:00,  7.60it/s]
Epoch 2/20	Loss: 0.3617	Metrics: 0.6666: 	Validation metrics: 0.5833: 	100% | 27/27 [00:00<00:00, 69.41it/s]
Epoch 3/20	Loss: 0.2019	Metrics: 0.7685: 	Validation metrics: 0.75: 	100% | 27/27 [00:00<00:00, 39.82it/s]
Epoch 4/20	Loss: 0.1377	Metrics: 0.8796: 	Validation metrics: 0.8333: 	100% | 27/27 [00:00<00:00, 35.71it/s]
Epoch 5/20	Loss: 0.0952	Metrics: 0.8611: 	Validation metrics: 0.9166: 	100% | 27/27 [00:00<00:00, 39.71it/s]
Epoch 6/20	Loss: 0.0581	Metrics: 0.8703: 	Validation metrics: 1.0: 	100% | 27/27 [00:00<00:00, 46.15it/s]
Epoch 7/20	Loss: 0.0372	Metrics: 0.8888: 	Validation metrics: 0.9166: 	100% | 27/27 [00:00<00:00, 35.34it/s]
Epoch 8/20	Loss: 0.0240	Metrics: 0.9259: 	Validation metrics: 0.9166: 	100% | 27/27 [00:00<00:00, 65.38it/s]
Epoch 9/20	Loss: 0.0159	Metrics: 0.9444: 	Validation metrics: 0.9166: 	100% | 27/27 [00:00<00:00, 50.94it/s]
Epoch 10/20	